In [1]:
import numpy as np
import pandas as pd 
import os
import sklearn 

In [2]:
print(os.listdir("../stat441 project/Resources/"))

['.DS_Store', 'test.csv', 'test_raw.csv', 'train_raw.csv', 'train.csv', 'codebook_compact.txt', 'codebook_long.txt', 'sample_submission.csv']


In [3]:
path = "../stat441 project/Resources/"

In [4]:
train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path+"test.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print('size of train_df data',train_df.shape) 
print('size of test_df data',test_df.shape) 

size of train_df data (30080, 273)
size of test_df data (9245, 272)


In [6]:
train_df.dtypes

id            int64
v1           object
v2           object
v3           object
v4           object
              ...  
v268         object
v269         object
v270         object
cntry        object
satisfied     int64
Length: 273, dtype: object

In [7]:
#Combine into data:
train_df['source']= 'train'
test_df['source'] = 'test'
data=pd.concat([train_df, test_df],ignore_index=True)
data.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


(39325, 274)

In [8]:
data.apply(lambda x: sum(x.isnull()))

cntry            0
id               0
satisfied     9245
source           0
v1               0
             ...  
v95           2097
v96           6972
v97          12820
v98              0
v99              0
Length: 274, dtype: int64

In [9]:
data.replace(['.a','.b','.c','.d'],np.NaN,inplace=True)
na_count = train_df.isnull().sum().sort_values(ascending=False)
na_rate = na_count/len(train_df)
na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
na_data.head(20)

,count,ratio
v123,27692,0.920612
v195,26907,0.894515
v89,26907,0.894515
v207,26907,0.894515
v262,26907,0.894515
v206,16037,0.533145
v261,16037,0.533145
v194,16037,0.533145
v88,16037,0.533145
v205,13200,0.438830


In [10]:
data = data.drop(na_data[na_data['ratio']>0.8].index,axis=1)
data.shape

(39325, 269)

In [11]:
data2 = data.drop(['id','satisfied','source'],axis=1)
quantity = [attr for attr in data2.columns if data2.dtypes[attr] != 'object'] 
quality = [attr for attr in data2.columns if data2.dtypes[attr] == 'object']

In [12]:
data[quantity].isnull().sum()

v106       0
v107       0
v129       0
v132    1574
v134       0
v15        0
v16        0
v175       0
v176       0
v185       0
v217       0
v218       0
v244       0
v245       0
v246       0
v247       0
v26        0
v27        0
v28        0
v29        0
v30        0
v31        0
v32        0
v33        0
v34        0
v36        0
v37        0
v38        0
v39        0
v40        0
v41        0
v42        0
v44        0
v45        0
v46        0
v47        0
v48        0
v49        0
v50        0
v51        0
v52        0
v54        0
v55        0
dtype: int64

In [13]:
quantity_df= data[quantity]
known_v132 = quantity_df[quantity_df.v132.notnull()]
unknown_v132 = quantity_df[quantity_df.v132.isnull()]
y_v132 = known_v132['v132'] 
X_v132 = known_v132.drop('v132',axis = 1)

In [14]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=0, n_estimators=2000,n_jobs=-1) 
rfr.fit(X_v132,y_v132) 
predicted_v132 = rfr.predict(unknown_v132.drop('v132',axis=1))
data.loc[(data.v132.isnull()),'v132'] = predicted_v132

In [15]:
small_missing = []
for c in quality:
    percent_missing = data[c].isnull().sum() * 100 / len(data[c])
    if percent_missing > 0 and percent_missing < 10:    
        small_missing.append(c)
print(small_missing) 

['v1', 'v10', 'v100', 'v101', 'v102', 'v103', 'v105', 'v108', 'v109', 'v110', 'v111', 'v112', 'v113', 'v114', 'v115', 'v116', 'v117', 'v118', 'v119', 'v12', 'v120', 'v121', 'v122', 'v124', 'v125', 'v126', 'v127', 'v128', 'v13', 'v130', 'v131', 'v133', 'v135', 'v136', 'v137', 'v138', 'v139', 'v14', 'v140', 'v141', 'v142', 'v143', 'v144', 'v145', 'v146', 'v147', 'v148', 'v149', 'v150', 'v152', 'v159', 'v162', 'v163', 'v165', 'v166', 'v167', 'v171', 'v172', 'v177', 'v178', 'v179', 'v18', 'v180', 'v181', 'v183', 'v184', 'v186', 'v187', 'v189', 'v19', 'v2', 'v21', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231', 'v232', 'v233', 'v234', 'v235', 'v236', 'v237', 'v238', 'v239', 'v24', 'v240', 'v242', 'v248', 'v249', 'v253', 'v254', 'v256', 'v257', 'v258', 'v3', 'v35', 'v4', 'v43', 'v53', 'v56', 'v57', 'v6', 'v64', 'v65', 'v7', 'v70', 'v72', 'v73', 'v74', 'v76', 'v77', 'v79', 'v8', 'v80', 'v81', 'v82', 'v83', 'v84', 'v85', 'v9', 'v98', 'v99

In [16]:
for c in small_missing:
    most_frequent = data[c].value_counts().keys()[0]
    data[c] = data[c].fillna(most_frequent)

In [17]:
large_missing = []
for c in quality:
    percent_missing = data[c].isnull().sum() * 100 / len(data[c])
    if percent_missing == 10 or percent_missing > 10 :    
        large_missing.append(c)
print(large_missing)

['v104', 'v11', 'v151', 'v153', 'v156', 'v157', 'v158', 'v160', 'v164', 'v168', 'v169', 'v170', 'v173', 'v174', 'v182', 'v188', 'v190', 'v191', 'v192', 'v193', 'v194', 'v196', 'v197', 'v198', 'v199', 'v200', 'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v22', 'v23', 'v241', 'v243', 'v250', 'v251', 'v252', 'v255', 'v259', 'v260', 'v261', 'v263', 'v264', 'v265', 'v266', 'v267', 'v268', 'v269', 'v270', 'v5', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v66', 'v67', 'v68', 'v69', 'v71', 'v75', 'v86', 'v87', 'v88', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97']


In [18]:
for c in large_missing:
    data[c] = data[c].fillna("MissValue")

In [19]:
data.apply(lambda x: sum(x.isnull()))

cntry           0
id              0
satisfied    9245
source          0
v1              0
             ... 
v95             0
v96             0
v97             0
v98             0
v99             0
Length: 269, dtype: int64

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for c in quality:
    data[c] = le.fit_transform(data[c].astype(str))

In [21]:
data.head()

,cntry,id,satisfied,source,v1,v10,v100,v101,v102,v103,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,0,9948,0.0,train,3,0,0,1,2,3,...,2,2,2,2,2,2,2,2,6,1
1,0,25601,0.0,train,5,0,0,1,3,4,...,2,2,2,2,2,2,2,2,5,2
2,0,8592,1.0,train,7,0,8,1,0,0,...,0,1,1,2,2,2,2,2,2,1
3,0,29593,0.0,train,2,0,0,1,0,0,...,2,2,2,2,2,2,2,2,9,0
4,0,4252,1.0,train,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,9,0


In [22]:
# separate train & test
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']
train.drop('source',axis=1,inplace=True)
test.drop(['source','satisfied'],axis=1,inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
df_X = train.drop(["id","satisfied"],axis=1)
df_y = train['satisfied']

In [24]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
tr = ExtraTreesClassifier()
fit = tr.fit(df_X, df_y) 

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
fit.feature_importances_
model = SelectFromModel(fit, prefit=True)
feature_idx = model.get_support()
feature_name = df_X.columns[feature_idx]

In [26]:
len(feature_name)

130

In [27]:
feature_name

Index(['cntry', 'v1', 'v101', 'v104', 'v105', 'v109', 'v110', 'v111', 'v112',
       'v113',
       ...
       'v79', 'v80', 'v81', 'v82', 'v83', 'v84', 'v85', 'v90', 'v98', 'v99'],
      dtype='object', length=130)

In [28]:
test.columns

Index(['cntry', 'id', 'v1', 'v10', 'v100', 'v101', 'v102', 'v103', 'v104',
       'v105',
       ...
       'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99'],
      dtype='object', length=267)

In [29]:
drop=[]
for i in test.columns:
    if i not in feature_name:
        if i != 'id':
            drop.append(i)
test = test.drop(drop,axis=1)

In [30]:
test.shape

(9245, 131)

In [31]:
drop=[]
for i in train.columns:
    if i not in feature_name:
        if i != 'id' and i != 'satisfied':
            drop.append(i)
train = train.drop(drop,axis=1)

In [32]:
train.shape

(30080, 132)

In [33]:
from sklearn.preprocessing import StandardScaler
train_test = train.drop("satisfied",axis=1)

In [34]:
train.to_csv("modified_train.csv", index=False)
test.to_csv("modified_test.csv", index=False)